# Logistic Regression

## Load Dependent Packages

In [1]:
import torch
import torchvision

## Select Hyper Parameters

In [2]:
# Hyper-parameters
input_size = 28 * 28
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.5

Download Dataset from Online:
```python
train_dataset = torchvision.datasets.MNIST(root='../data/MNIST', train=True, transform=torchvision.transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='../data/MNIST', train=False, transform=torchvision.transfroms.ToTensor(), download=True)
```

## Define Dataset Class

In [3]:
import numpy as np
import gzip
import os
class MNISTDataset(torch.utils.data.Dataset):
    def __init__(self, root, train=True, transform=None):
        # The file name prefix is obtained according to whether it is a training set or not.
        self.file_pre = 'train' if train == True else 't10k'
        self.transform = transform

        # Generate the image and label file path of the corresponding dataset.
        self.label_path = os.path.join(root, '%s-labels-idx1-ubyte.gz' % self.file_pre)
        self.image_path = os.path.join(root, '%s-images-idx3-ubyte.gz' % self.file_pre)

        # Read file data and return pictures and labels.
        self.images, self.labels = self.__read_data__(self.image_path, self.label_path)

    def __read_data__(self, image_path, label_path):
        # Data set reading.
        with gzip.open(label_path, 'rb') as lbpath:
            labels = np.frombuffer(lbpath.read(), np.uint8, offset=8)
        with gzip.open(image_path, 'rb') as imgpath:
            images = np.frombuffer(imgpath.read(), np.uint8, offset=16).reshape(len(labels), 28, 28)
        return images, labels

    def __getitem__(self, index):
        image, label = self.images[index], int(self.labels[index])
        
        # If you need to convert to tensor, use tansform.
        if self.transform is not None:
            image = self.transform(np.array(image))  # Avoid bug: use np.array
        return image, label

    def __len__(self):
        return len(self.labels)

## Load through Local Dataset

In [4]:
# MNIST dataset (images and labels)
train_dataset = MNISTDataset('../data/MNIST/', transform=torchvision.transforms.ToTensor())
test_dataset = MNISTDataset('../data/MNIST/', train=False, transform=torchvision.transforms.ToTensor())

## Set Data loader (Input Pipeline)

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

## Logistic Regression Model

In [6]:
model = torch.nn.Linear(input_size, num_classes)

In [7]:
print(model)

Linear(in_features=784, out_features=10, bias=True)


## Loss and Optimizer

In [8]:
criterion = torch.nn.CrossEntropyLoss() # nn.CrossEntropyLoss() computes softmax internally
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Train Model with CPU

In [9]:
total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, input_size)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_steps, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.5635
Epoch [1/5], Step [200/600], Loss: 0.3512
Epoch [1/5], Step [300/600], Loss: 0.2914
Epoch [1/5], Step [400/600], Loss: 0.2715
Epoch [1/5], Step [500/600], Loss: 0.3056
Epoch [1/5], Step [600/600], Loss: 0.1922
Epoch [2/5], Step [100/600], Loss: 0.3942
Epoch [2/5], Step [200/600], Loss: 0.4073
Epoch [2/5], Step [300/600], Loss: 0.3201
Epoch [2/5], Step [400/600], Loss: 0.4200
Epoch [2/5], Step [500/600], Loss: 0.2857
Epoch [2/5], Step [600/600], Loss: 0.2606
Epoch [3/5], Step [100/600], Loss: 0.3036
Epoch [3/5], Step [200/600], Loss: 0.2816
Epoch [3/5], Step [300/600], Loss: 0.3356
Epoch [3/5], Step [400/600], Loss: 0.2491
Epoch [3/5], Step [500/600], Loss: 0.1818
Epoch [3/5], Step [600/600], Loss: 0.2922
Epoch [4/5], Step [100/600], Loss: 0.4220
Epoch [4/5], Step [200/600], Loss: 0.2750
Epoch [4/5], Step [300/600], Loss: 0.3386
Epoch [4/5], Step [400/600], Loss: 0.3740
Epoch [4/5], Step [500/600], Loss: 0.3123
Epoch [4/5], Step [600/600], Loss:

## Test Model with no_grad

In [10]:
with torch.no_grad(): # no need to compute gradients (for memory efficiency)
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, input_size)
        outputs = model(images)
        # Find the maximum value of the prediction category probability of each row of samples, and 1 represents the row.(values, type)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    print ('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the model on the 10000 test images: 92.08000183105469 %


## Save the Model Checkpoint

In [11]:
torch.save(model.state_dict(), 'logistic_regression.ckpt')